In [2]:
%config IPCompleter.greedy = True

In [3]:
import numpy as np

import torch
from torch import nn, optim
import torch.nn.functional as F
from torchvision import datasets, transforms

import matplotlib as plt
%matplotlib inline

import helper

In [4]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_set = datasets.FashionMNIST('FashionMNIST_data/', download = True, train = True, transform = transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size = 8, shuffle = True)

In [5]:
test_set = datasets.FashionMNIST('FashionMNIST_data/', download = True, train = False, transform = transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size = 8, shuffle = True)

In [6]:
class Network(nn.Module):
    def __init__(self, input_layer, output_layer, hidden_layers, drop_p):
        super().__init__()
        
        self.hidden_layers = nn.ModuleList([nn.Linear(input_layer, hidden_layers[0])])
            
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes ])
        
        self.output_layer = nn.Linear(hidden_layers[-1], output_layer)
        
        self.dropout = nn.Dropout(p = drop_p)
        
        
    def forward(self, x):
        
        for linear in self.hidden_layers:
            x = F.relu(linear(x))
            x = self.dropout(x)
        
        x = self.output_layer(x)
        
        return F.log_softmax(x, dim = 1)

In [7]:
model = Network(784, 10, [516, 256], 0.5)
#print(model)

In [8]:
optimizer = optim.Adam(model.parameters(), lr = 0.001)
criterion = nn.NLLLoss()

In [9]:
def validation(model, test_loader, criterion):
    accuracy = 0
    test_loss = 0
    for images, labels in test_loader:
        
        images = images.resize_(images.size()[0], 784)
        
        output = model.forward(images)
        ps = torch.exp(output)
        
        test_loss += criterion(output, labels).item()
        
        equality = (labels.data == ps.max(dim = 1)[1])
        accuracy += equality.type(torch.FloatTensor).mean()
    
    return test_loss, accuracy

In [10]:
epochs = 2
steps = 0
print_every = 40
for e in range(epochs):
    running_loss = 0
    model.train()
    for images, labels in train_loader:
        steps +=1
        images.resize_(images.shape[0], 784)
        
        optimizer.zero_grad()
        
        output = model.forward(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        if steps % print_every == 0:
            model.eval()
            
            with torch.no_grad():
                test_loss, accuracy = validation(model, test_loader, criterion)
                print(
                    'running_loss : {:.3f}'.format(running_loss/print_every),
                    'test_loss : {:.3f}'.format(test_loss/len(test_loader)),
                    'accuracy {:.3f}'.format(accuracy/len(test_loader))
                )
                
            running_loss = 0
            model.train()

running_loss : 1.857 test_loss : 1.379 accuracy 0.490
running_loss : 1.280 test_loss : 1.045 accuracy 0.590
running_loss : 1.168 test_loss : 0.932 accuracy 0.665
running_loss : 1.061 test_loss : 0.859 accuracy 0.692
running_loss : 0.973 test_loss : 0.833 accuracy 0.709
running_loss : 1.007 test_loss : 0.807 accuracy 0.678
running_loss : 0.868 test_loss : 0.748 accuracy 0.714
running_loss : 0.891 test_loss : 0.739 accuracy 0.714
running_loss : 0.737 test_loss : 0.781 accuracy 0.721
running_loss : 0.902 test_loss : 0.694 accuracy 0.728
running_loss : 0.947 test_loss : 0.766 accuracy 0.701
running_loss : 0.815 test_loss : 0.695 accuracy 0.738
running_loss : 0.736 test_loss : 0.654 accuracy 0.751
running_loss : 0.717 test_loss : 0.659 accuracy 0.757
running_loss : 0.817 test_loss : 0.701 accuracy 0.722
running_loss : 0.890 test_loss : 0.671 accuracy 0.753
running_loss : 0.872 test_loss : 0.682 accuracy 0.739
running_loss : 0.787 test_loss : 0.688 accuracy 0.751
running_loss : 0.647 test_lo

running_loss : 0.692 test_loss : 0.512 accuracy 0.816
running_loss : 0.673 test_loss : 0.530 accuracy 0.808
running_loss : 0.609 test_loss : 0.503 accuracy 0.823
running_loss : 0.661 test_loss : 0.538 accuracy 0.790
running_loss : 0.626 test_loss : 0.521 accuracy 0.818
running_loss : 0.683 test_loss : 0.533 accuracy 0.806
running_loss : 0.621 test_loss : 0.494 accuracy 0.829
running_loss : 0.635 test_loss : 0.554 accuracy 0.799
running_loss : 0.717 test_loss : 0.517 accuracy 0.818
running_loss : 0.677 test_loss : 0.527 accuracy 0.811
running_loss : 0.616 test_loss : 0.545 accuracy 0.804
running_loss : 0.573 test_loss : 0.529 accuracy 0.813
running_loss : 0.579 test_loss : 0.527 accuracy 0.808
running_loss : 0.586 test_loss : 0.522 accuracy 0.810
running_loss : 0.674 test_loss : 0.532 accuracy 0.812
running_loss : 0.522 test_loss : 0.542 accuracy 0.804
running_loss : 0.688 test_loss : 0.518 accuracy 0.807
running_loss : 0.769 test_loss : 0.530 accuracy 0.815
running_loss : 0.642 test_lo

running_loss : 0.599 test_loss : 0.518 accuracy 0.809
running_loss : 0.668 test_loss : 0.493 accuracy 0.824
running_loss : 0.521 test_loss : 0.484 accuracy 0.823
running_loss : 0.632 test_loss : 0.491 accuracy 0.828
running_loss : 0.616 test_loss : 0.487 accuracy 0.825
running_loss : 0.682 test_loss : 0.497 accuracy 0.829
running_loss : 0.691 test_loss : 0.498 accuracy 0.825
running_loss : 0.530 test_loss : 0.496 accuracy 0.815
running_loss : 0.655 test_loss : 0.499 accuracy 0.812
running_loss : 0.608 test_loss : 0.491 accuracy 0.818
running_loss : 0.750 test_loss : 0.488 accuracy 0.822
running_loss : 0.564 test_loss : 0.487 accuracy 0.823
running_loss : 0.431 test_loss : 0.487 accuracy 0.818
running_loss : 0.692 test_loss : 0.483 accuracy 0.823
running_loss : 0.658 test_loss : 0.476 accuracy 0.828
running_loss : 0.558 test_loss : 0.476 accuracy 0.828
running_loss : 0.555 test_loss : 0.482 accuracy 0.821
running_loss : 0.626 test_loss : 0.531 accuracy 0.812
running_loss : 0.671 test_lo